# Enhancing HR Recruitment with MongoDB and OpenAI: A GraphRAG Approach

In [ ]:
!pip install --quiet pymongo dataset openai pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.


In [ ]:
import getpass
import os


# Function to securely get and set environment variables
def set_env_securely(var_name, prompt):
    value = getpass.getpass(prompt)
    os.environ[var_name] = value

## Data Loading and Preparation

In [ ]:
# load in csv with pandas
import pandas as pd

employee_df = pd.read_csv("employee_dataset_200.csv")

In [ ]:
import ast

# Convert 'skills' and 'certifications' to actual arrays (list format)
employee_df["skills"] = employee_df["skills"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)
employee_df["certifications"] = employee_df["certifications"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

In [ ]:
employee_df.head()

,employee_id,name,position,location,experience_years,employment_status,salary_range,team,skills,certifications,Experience,Summary
0,1,Jack Brown,DevOps Engineer,London,9,Contract,103k-208k,Frontend Development,"[SQL, Agile, React, CI/CD]",[Certified Frontend Developer],DevOps Engineer with 9 years of experience spe...,Jack Brown excels at delivering results in fro...
1,2,Hank Brown,DevOps Engineer,New York,3,Full-Time,150k-206k,Infrastructure,"[Data Analysis, Python, Docker, CSS]","[Oracle Certified, Google Cloud Certified]",DevOps Engineer with 3 years of experience spe...,Hank Brown excels at delivering results in inf...
2,3,Diana Taylor,Backend Developer,Dubai,14,Contract,140k-179k,Frontend Development,"[Python, Data Analysis, Java, API Development]",[Google Cloud Certified],Backend Developer with 14 years of experience ...,Diana Taylor excels at delivering results in f...
3,4,Frank Johnson,Backend Developer,San Francisco,9,Full-Time,148k-198k,Project Management,"[React, HTML, JavaScript]","[Oracle Certified, Certified Frontend Developer]",Backend Developer with 9 years of experience s...,Frank Johnson excels at delivering results in ...
4,5,Jack Taylor,Data Scientist,Paris,6,Full-Time,138k-207k,Backend Services,"[API Development, Java, Agile, Data Analysis, ...","[Certified Frontend Developer, Google Cloud Ce...",Data Scientist with 6 years of experience spec...,Jack Taylor excels at delivering results in ba...


In [ ]:
set_env_securely("OPENAI_API_KEY", "Enter your OPENAI API KEY: ")

Enter your OPENAI API KEY: ··········


In [ ]:
import openai


# Use OpenAI API to generate a summary for each data point
def summarize_datapoint(data_point):
    """
    Summarize the given data point using OpenAI's API.

    Args:
        data_point (str): The text to summarize.

    Returns:
        str: A concise summary of the input data.
    """
    # Ensure the input is valid
    if not data_point or not isinstance(data_point, str):
        raise ValueError("Invalid data point. Please provide a non-empty string.")

    try:
        # Call the OpenAI API
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": "You are an expert summarizer. Focus on the key points, removing unnecessary details. Write in a concise and clear manner.",
                },
                {
                    "role": "user",
                    "content": f"Please summarize the following data: {data_point}",
                },
            ],
        )

        # Extract the summary
        summary = response.choices[0].message.content

        return summary
    except Exception as e:
        return f"Error summarizing data: {e!s}"

In [ ]:
def create_datapoint_summary(row):
    """Concatenates all attributes of a row and generates a summary."""
    # All columns except 'datapoint_summary' should be concatenated
    attributes = [
        str(value) for key, value in row.items() if key != "datapoint_summary"
    ]
    data_point = " ".join(attributes)

    summary = summarize_datapoint(data_point)  # Call the summarization function
    return summary

In [ ]:
import tqdm

# Apply the function to create the 'datapoint_summary' column
employee_df["datapoint_summary"] = employee_df.apply(create_datapoint_summary, axis=1)

In [ ]:
employee_df.head()

,employee_id,name,position,location,experience_years,employment_status,salary_range,team,skills,certifications,Experience,Summary,datapoint_summary
0,1,Jack Brown,DevOps Engineer,London,9,Contract,103k-208k,Frontend Development,"[SQL, Agile, React, CI/CD]",[Certified Frontend Developer],DevOps Engineer with 9 years of experience spe...,Jack Brown excels at delivering results in fro...,Jack Brown is a DevOps Engineer in London with...
1,2,Hank Brown,DevOps Engineer,New York,3,Full-Time,150k-206k,Infrastructure,"[Data Analysis, Python, Docker, CSS]","[Oracle Certified, Google Cloud Certified]",DevOps Engineer with 3 years of experience spe...,Hank Brown excels at delivering results in inf...,Hank Brown is a Full-Time DevOps Engineer in N...
2,3,Diana Taylor,Backend Developer,Dubai,14,Contract,140k-179k,Frontend Development,"[Python, Data Analysis, Java, API Development]",[Google Cloud Certified],Backend Developer with 14 years of experience ...,Diana Taylor excels at delivering results in f...,Diana Taylor is a Backend Developer in Dubai w...
3,4,Frank Johnson,Backend Developer,San Francisco,9,Full-Time,148k-198k,Project Management,"[React, HTML, JavaScript]","[Oracle Certified, Certified Frontend Developer]",Backend Developer with 9 years of experience s...,Frank Johnson excels at delivering results in ...,Frank Johnson is a Backend Developer based in ...
4,5,Jack Taylor,Data Scientist,Paris,6,Full-Time,138k-207k,Backend Services,"[API Development, Java, Agile, Data Analysis, ...","[Certified Frontend Developer, Google Cloud Ce...",Data Scientist with 6 years of experience spec...,Jack Taylor excels at delivering results in ba...,Jack Taylor is a Data Scientist in Paris with ...


## Embedding Generation

In [ ]:
OPENAI_EMBEDDING_MODEL = "text-embedding-3-small"
OPENAI_EMBEDDING_MODEL_DIMENSION = 1536

In [ ]:
from tqdm import tqdm


# Generate an embedding using OpenAI's API
def get_embedding(text):
    """Generate an embedding for the given text using OpenAI's API."""

    # Check for valid input
    if not text or not isinstance(text, str):
        return None

    try:
        # Call OpenAI API to get the embedding
        embedding = (
            openai.embeddings.create(
                input=text,
                model=OPENAI_EMBEDDING_MODEL,
                dimensions=OPENAI_EMBEDDING_MODEL_DIMENSION,
            )
            .data[0]
            .embedding
        )
        return embedding
    except Exception as e:
        print(f"Error in get_embedding: {e}")
        return None

In [ ]:
# Apply the function to generate embeddings for all employees with error handling and progress tracking
try:
    employee_df["embedding"] = [
        x
        for x in tqdm(
            employee_df["datapoint_summary"].apply(get_embedding),
            total=len(employee_df),
        )
    ]
    print("Embeddings generated for employees")
except Exception as e:
    print(f"Error applying embedding function to DataFrame: {e}")

100%|██████████| 200/200 [00:00<00:00, 839700.50it/s]

Embeddings generated for employees


In [ ]:
employee_df.head()

,employee_id,name,position,location,experience_years,employment_status,salary_range,team,skills,certifications,Experience,Summary,datapoint_summary,embedding
0,1,Jack Brown,DevOps Engineer,London,9,Contract,103k-208k,Frontend Development,"[SQL, Agile, React, CI/CD]",[Certified Frontend Developer],DevOps Engineer with 9 years of experience spe...,Jack Brown excels at delivering results in fro...,Jack Brown is a DevOps Engineer in London with...,"[-0.033142998814582825, -0.016862226650118828,..."
1,2,Hank Brown,DevOps Engineer,New York,3,Full-Time,150k-206k,Infrastructure,"[Data Analysis, Python, Docker, CSS]","[Oracle Certified, Google Cloud Certified]",DevOps Engineer with 3 years of experience spe...,Hank Brown excels at delivering results in inf...,Hank Brown is a Full-Time DevOps Engineer in N...,"[-0.061603933572769165, -0.039547309279441833,..."
2,3,Diana Taylor,Backend Developer,Dubai,14,Contract,140k-179k,Frontend Development,"[Python, Data Analysis, Java, API Development]",[Google Cloud Certified],Backend Developer with 14 years of experience ...,Diana Taylor excels at delivering results in f...,Diana Taylor is a Backend Developer in Dubai w...,"[-0.025056499987840652, 0.0008755207527428865,..."
3,4,Frank Johnson,Backend Developer,San Francisco,9,Full-Time,148k-198k,Project Management,"[React, HTML, JavaScript]","[Oracle Certified, Certified Frontend Developer]",Backend Developer with 9 years of experience s...,Frank Johnson excels at delivering results in ...,Frank Johnson is a Backend Developer based in ...,"[-0.046922024339437485, 0.006648077629506588, ..."
4,5,Jack Taylor,Data Scientist,Paris,6,Full-Time,138k-207k,Backend Services,"[API Development, Java, Agile, Data Analysis, ...","[Certified Frontend Developer, Google Cloud Ce...",Data Scientist with 6 years of experience spec...,Jack Taylor excels at delivering results in ba...,Jack Taylor is a Data Scientist in Paris with ...,"[-0.06926461309194565, -0.003424275666475296, ..."


## Connecting to MongoDB

In [ ]:
# Set MongoDB URI
set_env_securely("MONGO_URI", "Enter your MONGO URI: ")

Enter your MONGO URI: ··········


In [ ]:
import pymongo


def get_mongo_client(mongo_uri):
    """Establish and validate connection to the MongoDB."""

    client = pymongo.MongoClient(
        mongo_uri, appname="devrel.showcase.rag.graphrag.employees.python"
    )

    # Validate the connection
    ping_result = client.admin.command("ping")
    if ping_result.get("ok") == 1.0:
        # Connection successful
        print("Connection to MongoDB successful")
        return client
    print("Connection to MongoDB failed")
    return None


MONGO_URI = os.environ["MONGO_URI"]
if not MONGO_URI:
    print("MONGO_URI not set in environment variables")

In [ ]:
mongo_client = get_mongo_client(MONGO_URI)

DB_NAME = "acme_corpration"
COLLECTION_NAME = "employees"

# Create or get the database
db = mongo_client[DB_NAME]

# Create or get the collections
collection = db[COLLECTION_NAME]

Connection to MongoDB successful


In [ ]:
collection.delete_many({})

DeleteResult({'n': 200, 'electionId': ObjectId('7fffffff0000000000000039'), 'opTime': {'ts': Timestamp(1732788089, 200), 't': 57}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1732788089, 200), 'signature': {'hash': b'\x8f\xc1\xcb\x89WD\xdeG+\x13u\xc5\xa0\xeeP\xba\xd1\x00\xc9\xd3', 'keyId': 7390008424139849730}}, 'operationTime': Timestamp(1732788089, 200)}, acknowledged=True)

## Data Ingestion

In [ ]:
documents = employee_df.to_dict("records")
collection.insert_many(documents)

print("Data ingestion into MongoDB completed")

Data ingestion into MongoDB completed


## MongoDB Graph Lookup

In [ ]:
# Updated GraphLookup Query: Find Employees with Shared Skills
graph_lookup_query = [
    {
        "$match": {"employee_id": 1}  # Start with Employee 1
    },
    {
        "$graphLookup": {
            "from": "employees",  # Collection name
            "startWith": "$team",  # Starting with the employee's skills array
            "connectFromField": "team",  # Match on array elements in the starting employee
            "connectToField": "team",  # Match on array elements in other employees
            "as": "related_employees",  # Output field for related employees
            "maxDepth": 1,  # Limit the depth of recursion
            "depthField": "level",  # Optional: Include the depth level in results
        }
    },
]

# Project the emebedding field
project_stage = {"$project": {"embedding": 0, "related_employees.embedding": 0}}

graph_lookup_query.append(project_stage)


# Execute the query
result = list(collection.aggregate(graph_lookup_query))

In [ ]:
import pprint

pprint.pprint(result)

[{'Experience': 'DevOps Engineer with 9 years of experience specializing in '
                'SQL, Agile, React, CI/CD.',
  'Summary': 'Jack Brown excels at delivering results in frontend development '
             'while leveraging skills in SQL, Agile, React, CI/CD.',
  '_id': ObjectId('67483f7c75040e5ce95c618a'),
  'certifications': ['Certified Frontend Developer'],
  'datapoint_summary': 'Jack Brown is a DevOps Engineer in London with 9 years '
                       'of experience. He specializes in SQL, Agile, React, '
                       'and CI/CD, and is a Certified Frontend Developer. He '
                       'excels in frontend development and is skilled in '
                       'various technologies.',
  'employee_id': 1,
  'employment_status': 'Contract',
  'experience_years': 9,
  'location': 'London',
  'name': 'Jack Brown',
  'position': 'DevOps Engineer',
  'related_employees': [{'Experience': 'Backend Developer with 3 years of '
                             

## Naive/Baseline RAG

In [ ]:
import time

from pymongo.operations import SearchIndexModel


def setup_vector_search_index(collection, index_definition, index_name="vector_index"):
    """
    Setup a vector search index for a MongoDB collection and wait for 30 seconds.

    Args:
    collection: MongoDB collection object
    index_definition: Dictionary containing the index definition
    index_name: Name of the index (default: "vector_index")
    """
    new_vector_search_index_model = SearchIndexModel(
        definition=index_definition, name=index_name, type="vectorSearch"
    )

    # Create the new index
    try:
        result = collection.create_search_index(model=new_vector_search_index_model)
        print(f"Creating index '{index_name}'...")

        # Sleep for 30 seconds
        print(f"Waiting for 30 seconds to allow index '{index_name}' to be created...")
        time.sleep(30)

        print(f"30-second wait completed for index '{index_name}'.")
        return result

    except Exception as e:
        print(f"Error creating new vector search index '{index_name}': {e!s}")
        return None

In [ ]:
vector_search_index_definition = {
    "fields": [
        {
            "type": "vector",
            "path": "embedding",
            "numDimensions": OPENAI_EMBEDDING_MODEL_DIMENSION,
            "similarity": "cosine",
        }
    ]
}

In [ ]:
setup_vector_search_index(
    collection, vector_search_index_definition, index_name="vector_index"
)

Creating index 'vector_index'...
Waiting for 30 seconds to allow index 'vector_index' to be created...
30-second wait completed for index 'vector_index'.


'vector_index'

In [ ]:
def vector_search(user_query, collection, vector_search_index_name="vector_index"):
    """
    Perform a vector search in the MongoDB collection based on the user query.

    Args:
    user_query (str): The user's query string.
    collection (MongoCollection): The MongoDB collection to search.
    additional_stages (list): Additional aggregation stages to include in the pipeline.
    vector_search_index_name (str): The name of the vector search index.

    Returns:
    list: A list of matching documents.
    """

    # Generate embedding for the user query
    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Define the vector search stage
    vector_search_stage = {
        "$vectorSearch": {
            "index": "vector_index",
            "queryVector": query_embedding,
            "path": "embedding",
            "numCandidates": 150,  # Number of candidate matches to consider
            "limit": 5,  # Return top 5 matches
        }
    }

    project_stage = {
        "$project": {
            "embedding": 0,  # Remove embedding from top-level documents
            "skills": 0,  # Remove skills from results
            "certifications": 0,  # Remove certifications from results
            "Summary": 0,  # Remove summary from results
        }
    }

    # Define the aggregate pipeline with the vector search stage and additional stages
    pipeline = [vector_search_stage, project_stage]

    # Execute the search
    results = collection.aggregate(pipeline)
    return list(results)

In [ ]:
def handle_user_query_naive_rag(query: str):
    results = vector_search(query, collection)

    # Pass query and results as input to openai
    if results:
        context = results

        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": "You are an expert in recommending teams based on employee data. Consider the provided employee data to answer the user's query. If the data is not sufficient to answer the query, simply state that you need more information.",
                },
                {
                    "role": "user",
                    "content": f"Here's the user's query: {query}\n\nHere's some potentially relevant employee data: {context}",
                },
            ],
        )

        # Extract and return the OpenAI's response
        return response.choices[0].message.content
    return "No relevant employees found for this query."

In [ ]:
query = (
    "Get me employees that can form a team to build a website for HR recruitement firm"
)

In [ ]:
naive_rag_results = handle_user_query_naive_rag(query)

In [ ]:
print(naive_rag_results)

Based on the provided employee data, I can suggest a team to build a website for an HR recruitment firm. A typical website development team might require roles such as a frontend developer, a backend developer, and a project manager.

1. **Frontend Developers**:
   - **Ivy Hall**: With 15 years of experience, Ivy specializes in React and CSS, which are crucial for frontend development.
   - **Frank Hall**: With 11 years of experience, specializing in Data Analysis and Agile methods, can contribute to the frontend design and functionality, especially in implementing agile practices.
   - **Hank Clark**: Although less experienced, he brings skills in SQL, Agile, and Data Analysis, which can be valuable for integrating frontend functionalities with data operations.

2. **Project Manager**:
   - **Grace Hall**: With 7 years of experience and skills in SQL, CI/CD, and Data Analysis, Grace can manage the project effectively, ensuring seamless communication and integration in a cross-function

## GraphRAG

In [ ]:
def customGraphRAG(text: str):
    """
    Performs a custom GraphRAG operation by conducting a vector search
    followed by graph traversal using graphLookup.

    Args:
        text (str): The query text.

    Returns:
        list: A list of documents containing the relevant results.
    """

    # Step 1: Generate the embedding for the query text
    query_embedding = get_embedding(text)

    # Step 2: Define the vector search pipeline
    vector_search_stage = {
        "$vectorSearch": {
            "index": "vector_index",
            "queryVector": query_embedding,
            "path": "embedding",
            "numCandidates": 150,  # Number of candidate matches to consider
            "limit": 1,  # Return top 5 matches
        }
    }

    # Step 3: Define the graph traversal pipeline
    graph_lookup_stage = {
        "$graphLookup": {
            "from": "employees",  # Collection to perform graph traversal on
            "startWith": "$skills",  # Start the traversal using skills field
            "connectFromField": "skills",  # Field in the current document to match
            "connectToField": "skills",  # Field in the other documents to match
            "as": "related_employees",  # Output field for connected documents
            "maxDepth": 2,  # Depth of graph traversal
            "depthField": "level",  # Include recursion level in results
        }
    }

    # Step 4: Exclude embeddings from the output (optional cleanup)
    project_stage = {
        "$project": {
            "_id": 1,
            "embedding": 0,  # Remove embedding from top-level documents
            "related_employees.embedding": 0,  # Remove embedding from nested results
            "related_employees.skills": 0,  # Remove skills from nested results
            "related_employees.certifications": 0,  # Remove certifications from nested results
            "related_employees.Summary": 0,  # Remove summary from nested results
        }
    }

    # Step 5: Combine the stages into a pipeline
    pipeline = [
        vector_search_stage,  # Perform vector search
        graph_lookup_stage,  # Conduct graph traversal
        project_stage,  # Clean up unnecessary fields
    ]

    # Step 6: Execute the aggregation pipeline
    try:
        result = list(collection.aggregate(pipeline))
        return result
    except Exception as e:
        print(f"An error occurred: {e!s}")
        return []

In [ ]:
results = customGraphRAG(query)

In [ ]:
pprint.pprint(results)

### Prompt Compression Technique
Because GraphRAG can be expensive, token wise

In [ ]:
!pip install --quiet -U llmlingua

In [ ]:
from llmlingua import PromptCompressor

# https://github.com/microsoft/LLMLingua
llm_lingua = PromptCompressor(
    model_name="microsoft/llmlingua-2-xlm-roberta-large-meetingbank",  # Smaller Model: microsoft/llmlingua-2-bert-base-multilingual-cased-meetingbank
    use_llmlingua2=True,
    device_map="cpu",
)

config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [ ]:
def handle_user_query(query: str, compress_prompt=False):
    results = customGraphRAG(query)

    # Pass query and results as input to openai
    if results and results[0].get("related_employees"):
        context = results[0]["related_employees"]

        prompt = f"Here's the user's query: {query}\n\nHere's some potentially relevant employee data: {context}"

        if compress_prompt:
            compression_result = llm_lingua.compress_prompt(
                prompt, rate=0.20, force_tokens=["\n", "?"]
            )
            # Uncomment below to see the compression results
            # pprint.pprint(compression_result)
            final_prompt = compression_result["compressed_prompt"]
            print(f"Compressed prompt: {final_prompt}")
            print()
        else:
            final_prompt = prompt

        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": "You are an expert in recommending teams based on employee data. Consider the provided employee data to answer the user's query.",
                },
                {"role": "user", "content": final_prompt},
            ],
        )

        # Extract and return the OpenAI's response
        return response.choices[0].message.content
    return "No relevant employees found for this query."

In [ ]:
results = handle_user_query(query)

print(results)

To form a team to build a website for an HR recruitment firm, you would ideally look for a mix of frontend developers, backend developers, a project manager, and potentially a data scientist or DevOps engineer for infrastructure and data needs. Here are some recommended candidates from the provided data:

1. **Charlie Wilson** - Project Manager (Dubai)
   - Experience: 7 years in project management
   - Skills: Specializing in CSS, JavaScript
   - Team: Project Management

2. **Bob Brown** - Frontend Developer (Toronto)
   - Experience: 11 years
   - Skills: Agile, CSS, JavaScript, Data Analysis
   - Team: Project Management

3. **Jack Hall** - Backend Developer (Sydney)
   - Experience: 5 years
   - Skills: React, SQL, Kubernetes
   - Team: Project Management

4. **Grace Taylor** - Frontend Developer (Berlin)
   - Experience: 6 years
   - Skills: Machine Learning, HTML, CSS, Agile, CI/CD
   - Team: Frontend Development

5. **Ivy Taylor** - Project Manager (London)
   - Experience: 13 

In [ ]:
results = handle_user_query(query, compress_prompt=True)

print(results)

Token indices sequence length is longer than the specified maximum sequence length for this model (19789 > 512). Running this sequence through the model will result in indexing errors


Compressed prompt: user query employees website HR

 employee data Brown Engineer York_years 3-Time-206k 3 Data Analysis Python Docker Brown Data Analysis Python Docker CSS Oracle Google Cloud Certified Python Docker CSS annual salary 150k to 206k Brown Developer 11-Time-173k Management Developer 11 Agile CSS SQL JavaScriptskilled Project Management Certified Frontend Developer PMP Certified results Wilson Scientist 9 Development Agile Kubernetes React JavaScript Scrum Wilson Scientist Agile Kubernetes React JavaScript Scrum frontend development certifications Oracle PMP Google Cloud Wilson Manager 7-Time-177k Management 7 CSS JavaScript resultsHall Developer 5-Time-179k Management Developer 5 years React SQL Kubernetes Part-Time Backend Developer 5 React SQL Kubernetes project Frontend Developer AWS Certified Johnson Developer 15-152k 15 CSS JavaScript Agile CSS JavaScript Agile PMP Certified Google Cloud CertifiedDavis Manager York 14-Time-202k Development 14 Data Analysis React Mach